In [1]:
import pandas as pd
import os
import scipy.stats as stats

In [2]:
carpetas = ['GUROBI','CPLEX','MLM','ESGH','LPDH','Versiones']

In [3]:
gur = []
mlm = []
esgh = []
lpdh = []
cplex = []
versiones = []

for c in carpetas:
    archivos = os.listdir(c)
    for ar in archivos: 
        # print(sorted(ar.rstrip(".xlsx").split("_")))
        # print(sorted(ar.rstrip(".xlsx").split("_")))
        tiempo, solver, capacidad, nodos  = sorted(ar.rstrip(".xlsx").split("_"))
        tiempo = int(tiempo)
        capacidad = int(capacidad[1:])
        nodos = int(nodos[1:])
        datos = pd.read_excel(f"{c}/{ar}")
        datos["Q"] = capacidad
        datos["nodos"] = nodos
        datos["tiempos"] = tiempo
        if solver == "GUROBI":
            gur.append(datos)
        if solver == "CPLEX":
            cplex.append(datos)
        elif solver == "MLM":
            mlm.append(datos)
        elif solver == "ESGH":
            esgh.append(datos)
        elif solver == "LPDH":
            lpdh.append(datos)
        elif solver in ["Expv1", "Expv2", "Expv3", "Expv4"]:
            datos["version"] = "Version" + solver[-1]
            versiones.append(datos)
        else: pass
        
gurobis = pd.concat(gur)
cplexs  = pd.concat(cplex)
mlms    = pd.concat(mlm)
esghs   = pd.concat(esgh)
lpdhs   = pd.concat(lpdh)
versiones   = pd.concat(versiones)

In [4]:
gurobis[["LB","UB"]] = gurobis[["LB","UB"]].round(2)
cplexs[["LB","UB"]] = cplexs[["LB","UB"]].round(2)
mlms[["min", "avg"]] = mlms[["min", "avg"]].round(2)

In [7]:
bks = pd.concat((
    mlms.groupby(["name", "Q", "nodos"]).agg(bks = ("min", "min")).reset_index(), 
    gurobis.groupby(["name", "Q", "nodos"]).agg(bks = ("UB", "min")).reset_index(),
    cplexs.groupby(["name", "Q", "nodos"]).agg(bks = ("UB", "min")).reset_index(),
    esghs.groupby(["name", "Q", "nodos"]).agg(bks = ("cost", "min")).reset_index(),
    lpdhs.groupby(["name", "Q", "nodos"]).agg(bks = ("cost", "min")).reset_index(),
    versiones.groupby(["name", "Q", "nodos"]).agg(bks = ("min", "min")).reset_index(),
))

bks = bks.groupby(["name", "Q", "nodos"]).agg(bks = ("bks", "min")).reset_index()

gurobis = gurobis.merge(bks, on = ["name", "Q", "nodos"])
cplexs  = cplexs.merge(bks, on = ["name", "Q", "nodos"] )
mlms    = mlms.merge(bks, on = ["name", "Q", "nodos"])
esghs   = esghs.merge(bks, on = ["name", "Q", "nodos"])
lpdhs   = lpdhs.merge(bks, on = ["name", "Q", "nodos"])
versiones = versiones.merge(bks, on = ["name", "Q", "nodos"])

In [8]:
gurobis["best"] = gurobis["UB"]
cplexs["best"] = cplexs["UB"]
mlms["best"] = mlms["min"]
versiones["best"] = versiones["min"]
lpdhs["best"] = lpdhs["cost"]

gurobis["gap_best"]    = (gurobis["UB"] - gurobis["bks"]) / gurobis["bks"]
cplexs["gap_best"]     = (cplexs["UB"] - cplexs["bks"]) / cplexs["bks"]
mlms["gap_best"]       = (mlms["min"] - mlms["bks"]) / mlms["bks"]
mlms["gap_avg"]        = (mlms["avg"] - mlms["bks"]) / mlms["bks"]
versiones["gap_best"]  = (versiones["min"] - versiones["bks"]) / versiones["bks"]
versiones["gap_avg"]   = (versiones["avg"] - versiones["bks"]) / versiones["bks"]
lpdhs["gap"]           = (lpdhs["cost"] - lpdhs["bks"]) / lpdhs["bks"]
esghs["gap"]           = (esghs["cost"] - esghs["bks"]) / esghs["bks"]

In [9]:
def wilcoxon(gurobis, mlms, base = "gap_best", 
             tiempos = [60, 120, 180], nodos = [100, 150, 200]):
    global exp1, exp2, exp3
    tabla = []
    for n in nodos:
        fila = dict()
        fila["tamaño"] = n
        for t in tiempos:
            exp1 = mlms[(mlms.tiempos == t) & (mlms.nodos == n)][base]
            exp2 = gurobis[(gurobis.tiempos == t) & (gurobis.nodos == n)][base]
            fila[f"gurobi {t}"] = stats.wilcoxon(exp1, exp2, alternative = "less").pvalue
            try:
                exp3 = gurobis[(gurobis.tiempos == 3600) & (gurobis.nodos == n)][base]
                fila[f"gurobi benckmark-{t}"] = stats.wilcoxon(exp1, exp3, alternative = "less").pvalue
            except: pass
        tabla.append(fila)
    return pd.DataFrame(tabla)
    
tabla = wilcoxon(gurobis, mlms)
tabla = tabla[sorted(tabla.columns)]

In [10]:
tabla2 = tabla.set_index("tamaño")
tabla2

,gurobi 120,gurobi 180,gurobi 60,gurobi benckmark-120,gurobi benckmark-180,gurobi benckmark-60
tamaño,,,,,,
100,NaN,NaN,6.076579e-08,NaN,NaN,1.0
150,5.587255e-36,2.304942e-33,5.091616e-14,1.0,1.0,1.0
200,2.487865e-42,4.421510e-24,1.096320e-10,1.0,1.0,1.0


In [11]:
def wilcoxon2(gurobis, mlms, base = "gap_best", 
             tiempos = [60, 120, 180], nodos = [100, 150, 200]):
    global exp1, exp2, exp3
    tabla = []
    for n in nodos:
        fila = dict()
        fila["tamaño"] = n
        for t in tiempos:
            exp1 = mlms[(mlms.tiempos == t) & (mlms.nodos == n)][base]
            exp2 = gurobis[(gurobis.tiempos == t) & (gurobis.nodos == n)][base]
            fila[f"gurobi {t}"] = stats.wilcoxon(exp1, exp2, alternative = "less").pvalue
        tabla.append(fila)
    return pd.DataFrame(tabla)
    
tabla = wilcoxon2(gurobis, mlms)
tabla = tabla[sorted(tabla.columns)]

tabla2 = tabla.set_index("tamaño")
tabla2.columns = ["120s", "180s", "60s"]
tabla2 = tabla2[["60s","120s", "180s"]].reset_index()
tabla2.columns = ["instance Size","60s","120s", "180s"]
latex = tabla2.to_latex(
    index = False,
    column_format = "c ccc",
    multicolumn_format = "c",
    multirow = True, 
    caption =  "p-values of the Wilcoxon test performed over the experiments",
    label = "tab:wilcoxon",
    position = "H",
    na_rep = "-",
    escape = False,
    float_format = '{:.2E}'.format)

print(latex)
with open("latex/wilcoxon.tex", "a") as file:
    file.write(latex)

\begin{table}[H]
\centering
\caption{p-values of the Wilcoxon test performed over the experiments}
\label{tab:wilcoxon}
\begin{tabular}{c ccc}
\toprule
 instance Size &      60s &     120s &     180s \\
\midrule
           100 & 6.08E-08 &        - &        - \\
           150 & 5.09E-14 & 5.59E-36 & 2.30E-33 \\
           200 & 1.10E-10 & 2.49E-42 & 4.42E-24 \\
\bottomrule
\end{tabular}
\end{table}



C:\Users\rablo\AppData\Local\Temp\ipykernel_23284\325675047.py:22: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = tabla2.to_latex(
